## Cartpole using heuristics

Read more here: https://medium.com/@twocolossi/brute-forcing-the-cartpole-problem-4d04c9c34b12

In [3]:
import gymnasium as gym
import numpy as np
import time

### Policy Generator

To brute force the problem, we discretise the state space. The 'Pole Angle' and 'Pole Velocity At Tip' are both split into 3 buckets creating 9 possible states of the environment (we ignore the cart position and velocity observations). Cartpole only has 2 actions, so with 9 states, we have 512 (2^9) deterministic greedy policies (A Policy that will always pick the same one action given the same state). 

To create the policies, we convert the numbers 0 to 511 to binary and reshape them to a 3 by 3 matrix. 

In [4]:
def createPolicy(id):
    binary = unpackbits(np.array([id]), 9)
    return np.reshape(binary, (3,3))

#Credit for this function https://stackoverflow.com/a/51509307
def unpackbits(x, num_bits):
          xshape = list(x.shape)
          x = x.reshape([-1,1])
          to_and = 2**np.arange(num_bits).reshape([1,num_bits])
          return (x & to_and).astype(bool).astype(int).reshape(xshape + [num_bits])

### Create a class to discretise the observation space
Note: this is used in other exercises with CARTPOLE

In [5]:
class DiscreteBox(object):
    def __init__(self, low, high, shape):
        self.low, self.high, self.shape = low, high, shape

    def Discretise(self, state):   
        discreteState = [int(np.floor((state[i] - self.low[i])/(self.high[i]-self.low[i])*(self.shape[i]-1))) for i in range(len(state))]
        return tuple([np.min([self.shape[i]-1, np.max([discreteState[i], 0])]) for i in range(len(state))])

### Create environment and discretiser

We have set the bounds for discrete space tighter than that seen in the observation space. We have a very limited amount of buckets so we need one of the buckets to be in a stable area of the state space. 


In [6]:
env = gym.make('CartPole-v1')

thetaHigh = 10 * 2 * np.pi / 360
high = np.array([thetaHigh, np.radians(15)])
observationSpace = DiscreteBox(-high, high, (3,3))

### Brute Force Algorithm

As the number of possible policies is finite we try them to identify the right ones. 

In [7]:
startTime = time.time()
resample = []
for i in range(512):
    state, _ = env.reset()
    policy = createPolicy(i)
    step = 0
    while True:
        step += 1
        state = observationSpace.Discretise(state[2:])
        action = policy[state]
        state, r, terminal, _, info = env.step(action)
        if terminal or step >= 200:
            if step > 195:
                resample.append(i)
            break

print(str(len(resample)) + ' potential solutions found in ' + "{:.1f}".format(time.time()-startTime) + ' seconds.')

24 potential solutions found in 0.5 seconds.


### Find solutions

We test the selected candidate solution for 100 episodes. If they average over 195 reward they are solutions to the cartpole problem.

In [10]:
startTime, solutionCount = time.time(), 0
for i in resample:
    avg = 0
    for k in range(100):
        state, _ = env.reset()
        policy = createPolicy(i)
        step = 0
        while True:
            step += 1
            state = observationSpace.Discretise(state[2:])
            action = policy[state]
            state, r, terminal, _, info = env.step(action)
            if terminal or step >= 200:
                avg += step
                break
    if avg/100 >= 195:
        print("Solution at Index: " + str(i) + " , score: " + str(avg/100))
        solutionCount += 1

print(str(solutionCount) + ' solutions found in ' + "{:.1f}".format(time.time()-startTime) + ' seconds.')

Solution at Index: 60 , score: 200.0
Solution at Index: 124 , score: 200.0
Solution at Index: 188 , score: 200.0
Solution at Index: 252 , score: 200.0
Solution at Index: 316 , score: 200.0
Solution at Index: 380 , score: 200.0
Solution at Index: 444 , score: 200.0
Solution at Index: 508 , score: 200.0
8 solutions found in 7.7 seconds.


8 solutions found at about 5 seconds per solution. That is pretty competitive for solve times. 

Now, clearly, this is not a good method to solve the cartpole problem. The solutions are very unstable, none of them could balance the pole for more than 300 timesteps.  Yet, they are valid solutions. 